### Import dependencies

In [27]:
from db import postgres
from os import environ
import pandas as pd
import sqlalchemy
from dotenv import load_dotenv
load_dotenv()

True

### Connect to Database

In [28]:
user = environ.get('POSTGRES_USER')
password = environ.get('POSTGRES_PASSWORD')
host = environ.get('POSTGRES_HOST')
database = environ.get('POSTGRES_DATABASE_NAME')

conn_string = f'postgresql://{user}:{password}@{host}/{database}'
engine = sqlalchemy.create_engine(conn_string)
connection = engine.connect()
metadata = sqlalchemy.MetaData()
abc_articles = sqlalchemy.Table('abc_article', metadata, autoload=True, autoload_with=engine)

### Basic "Select *" query.

Using DataFrame to display better.

In [29]:
query = sqlalchemy.select([abc_articles]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
select_all_df = pd.DataFrame(ResultSet)
select_all_df.head(2)

,id,title,link,author,description,published,last_updated,tags,content,key_points,title_sentiment,key_points_sentiment
0,89,The cost-of-living debate in this election cam...,/news/2022-04-30/election-campaign-cost-of-liv...,[Laura Tingle],We need to be able to have a proper conversati...,2022-04-29 19:00:00+00:00,2022-04-29 21:11:30+00:00,"[election, federal election, ndis, people with...",Go to the website of Australia's Productivity ...,None,-0.4019,None
1,92,"After a month, Lidia Zaika came out of hiding ...",/news/2022-04-30/kyiv-cleans-up-after-russian-...,[Nick Dole],With Russian troops now seeking victory for Vl...,2022-04-29 18:56:24+00:00,NaT,"[Kyiv, Ukraine, Vladimir Putin, Russia, atroci...",Lidia Zaika heard the Russians coming well bef...,None,-0.296,None


### Number of articles in Database

In [39]:
print(f"Count of articles: {select_all_df.id.count()}")

Count of articles: 200


### Number of unique authors for the articles

In [36]:
sql_query = """
WITH authors as (
	SELECT UNNEST(author) as author
	FROM abc_article
)
SELECT COUNT(DISTINCT(author))
FROM authors;
"""

res = connection.execute(sql_query)

pd.DataFrame(res)

,count
0,144


### Top 10 writing authors

In [44]:
sql_query = """
WITH authors as (
	SELECT UNNEST(author) as author
	FROM abc_article
)
SELECT author, COUNT(author) as articles_written
FROM authors
GROUP BY author
ORDER BY articles_written DESC
LIMIT 10;
"""

res = connection.execute(sql_query)

pd.DataFrame(res)

,author,articles_written
0,Melinda James,2
1,Will Jackson,2
2,Iris Zhao,2
3,Madeleine Cross,2
4,Luke Radford,2
5,Emma Haskin,2
6,Hinako Shiraishi,2
7,Stephen Dziedzic,2
8,Nicholas McElroy,2
9,Brooke Neindorf,2


### Author sentiment analysis

In [57]:
sql_query = """
WITH authors as (
	SELECT UNNEST(author) as author
	FROM abc_article
),
author_sentiment as (
	SELECT UNNEST(author) as author, 
		   ROUND(title_sentiment, 4) as title_sentiment,
		   ROUND(key_points_sentiment, 4) as key_points_sentiment
	FROM abc_article
)

SELECT author_sentiment.author, 
	   COUNT(author_sentiment.author) as articles_written, 
	   title_sentiment,
	   key_points_sentiment,
	   abs(title_sentiment - key_points_sentiment) as "sentiment_difference",
FROM authors
JOIN author_sentiment
	ON author_sentiment.author = authors.author
WHERE abs(title_sentiment - key_points_sentiment) IS NOT NULL 
GROUP BY author_sentiment.author, title_sentiment, key_points_sentiment
ORDER BY sentiment_difference DESC
"""

res = connection.execute(sql_query)

pd.DataFrame(res).head(10)

,author,articles_written,title_sentiment,key_points_sentiment,sentiment_difference
0,Elizabeth Wright,1,-0.7906,0.2164,1.0070
1,Celina Edmonds,1,-0.7906,0.2164,1.0070
2,Evan Young,1,-0.7906,0.2164,1.0070
3,Anne Connolly,1,-0.7626,0.1625,0.9251
4,Sarah Curnow,1,-0.7626,0.1625,0.9251
5,Alicia Nally,1,-0.7184,0.1044,0.8228
6,Hinako Shiraishi,2,-0.6486,0.0593,0.7079
7,Jamie McKinnell,1,-0.8658,-0.2041,0.6617
8,Kristian Silva,1,0.0772,-0.5642,0.6414
9,Leanne Wong,1,0.0772,-0.5642,0.6414


### Number of unique tags used for the articles

In [59]:
sql_query = """
WITH tags as (
	SELECT UNNEST(tags) as tag
	FROM abc_article
)
SELECT COUNT(DISTINCT(tag))
FROM tags;
"""

res = connection.execute(sql_query)

pd.DataFrame(res)

,count
0,1318


### Tag sentiment analysis

In [79]:
sql_query = """
WITH tags as (
	SELECT UNNEST(tags) as tag,
		   ROUND(title_sentiment, 4) as title_sentiment,
		   ROUND(key_points_sentiment, 4) as key_points_sentiment
	FROM abc_article
)
SELECT tag, 
	   COUNT(tag) as uses, 
	   ROUND(AVG(title_sentiment), 4) as avg_title_sentiment,
	   ROUND(AVG(key_points_sentiment), 4) as avg_key_points_sentiment,
	   ROUND(ABS(AVG(title_sentiment) - AVG(key_points_sentiment)), 4) as "sentiment_difference"
FROM tags
WHERE tag != ''
GROUP BY tag
ORDER BY uses DESC
"""

res = connection.execute(sql_query)

pd.DataFrame(res).head(10)

,tag,uses,avg_title_sentiment,avg_key_points_sentiment,sentiment_difference
0,federal election,15,-0.0764,0.0460,0.1224
1,coronavirus,11,0.0609,0.0219,0.0390
2,sport,11,0.0253,0.1582,0.1329
3,election,10,0.1283,0.0160,0.1123
4,court,9,-0.2570,-0.1619,0.0951
5,covid-19,9,-0.1764,0.0729,0.2494
6,covid,9,-0.1078,-0.0669,0.0409
7,northern territory,9,-0.0431,-0.0759,0.0328
8,adelaide,7,-0.1877,-0.0178,0.1699
9,ukraine,7,-0.2178,-0.1321,0.0857
